In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

2025-07-12 10:59:40.430579: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-12 10:59:40.546849: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-12 10:59:40.546997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-12 10:59:40.550703: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-12 10:59:40.570293: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-12 10:59:40.571745: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import ssl
import certifi

ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())

In [4]:
import keras
model = keras.saving.load_model('simple_rnn_imdb.keras')

#model=load_model('simple_rnn_imdb.h5')
model.summary()

2025-07-12 11:00:55.950957: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model.get_weights()

[array([[ 1.3273010e-02, -6.1035785e-03,  5.3774569e-02, ...,
          6.0545966e-02, -6.1385550e-02,  3.5275243e-02],
        [ 1.7034316e-02,  8.8004768e-03,  3.8059916e-02, ...,
         -4.1362796e-02, -1.6612850e-02, -2.0490962e-03],
        [ 4.2631354e-02,  9.0043498e-03, -2.5913797e-02, ...,
          6.4967773e-03, -7.8642920e-02, -1.9447235e-02],
        ...,
        [-3.7984293e-02, -5.4470189e-02,  8.9507128e-05, ...,
          2.0977868e-02,  8.8861007e-03, -6.9489703e-02],
        [ 3.3350278e-02,  1.9911312e-02,  1.8570069e-02, ...,
          1.0940048e-02, -1.6037000e-02, -7.0262045e-02],
        [ 7.1893777e-03, -2.4584837e-02,  3.4378592e-02, ...,
          8.2182707e-03, -2.0011129e-02, -7.3209717e-03]], dtype=float32),
 array([[ 0.01070341,  0.02598631,  0.00574371, ...,  0.14648414,
          0.11211056, -0.1074672 ],
        [ 0.1126101 , -0.06684127,  0.08868878, ..., -0.07869948,
         -0.04825716,  0.08819304],
        [ 0.12785292, -0.14620116, -0.07136921

In [6]:
word_index=imdb.get_word_index()
reverse_word_index={value:key for key, value in word_index.items()}

In [7]:
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?') for i in encoded_review])

#function to preprocess user input
def preprocess_text(text):
    words=text.lower().split()
    encoded_review=[word_index.get(word,2)+3 for word in words]
    padded_review=sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

In [8]:
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [9]:
##prediction function
def predict_sentiment(review):
    processed_input=preprocess_text(review)

    prediction=model.predict(processed_input)

    sentiment='Positive' if prediction[0][0] >0.5 else 'Negetive'

    return sentiment,prediction[0][0]

In [10]:
example_review="this movie is fantastic! the acting was great and plot was thrilling"

sentiment,score=predict_sentiment(example_review)

1/1 [==============================] - 0s 471ms/step


In [11]:
print(f'review : {example_review}')
print(f'sentiment : {sentiment}' )
print(f'prediction score : {score}')

review : this movie is fantastic! the acting was great and plot was thrilling
sentiment : Negetive
prediction score : 0.3928641080856323
